In [5]:
%pip install --upgrade pip
%pip install transformers
%pip install gensim
%pip install spacy #rather use Terminal with:
%pip install -U spacy
%pip install nltk

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

In [6]:
# Step 1 importing packages
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
from transformers import BertTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
#nlp = spacy.load('en_core_web_sm')
from utils import remove_similar_rows

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevingiesen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import spacy
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [52]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)
df.head()


,data,player,language,publishedAt
0,"{'content': ""DIRECTO\nMercado de fichajes de f...",Exequiel Palacios,es,2023-01-29T18:25:03Z
1,{'content': 'Con el primer mes del 2023 a poco...,Exequiel Palacios,es,2023-01-30T16:52:46Z
2,{'content': 'Deportes\nGustavo Puerta ya no ju...,Exequiel Palacios,es,2023-01-31T20:41:38Z
3,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-09T18:32:38Z
4,{'content': 'Dólar blue\nAlberto Fernández\nMa...,Exequiel Palacios,es,2023-02-12T21:13:55Z


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         1104 non-null   object
 1   player       1104 non-null   object
 2   language     1104 non-null   object
 3   publishedAt  1104 non-null   object
dtypes: object(4)
memory usage: 34.6+ KB


In [54]:
# Match the two different spellings of Hincapié i choose Hincapie for easier processing
df.loc[df['player'] == 'Piero Hincapié', 'player'] = 'Piero Hincapie'

In [56]:
# Remove the similiar rows (The Function is imported from utils on top)
df = remove_similar_rows(pd.DataFrame(df['data']), df)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 731 entries, 0 to 1103
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         731 non-null    object
 1   player       731 non-null    object
 2   language     731 non-null    object
 3   publishedAt  731 non-null    object
dtypes: object(4)
memory usage: 28.6+ KB


In [58]:
df['player'].value_counts()

player
Exequiel Palacios    182
Jeremie Frimpong     151
Moussa Diaby         118
Piero Hincapie        96
Mykhaylo Mudryk       93
Jonathan Tah          46
Mitchel Bakker        45
Name: count, dtype: int64

In [59]:
df['language'].value_counts()
#for more about that, pls see "Language_check.ipynb"

language
de    275
en    261
es    195
Name: count, dtype: int64

## Make data in small letters

In [60]:
data_lower = df.copy()

In [61]:
data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

,data,player,language,publishedAt
0,"{'content': ""directo\nmercado de fichajes de f...",exequiel palacios,es,2023-01-29T18:25:03Z
1,{'content': 'con el primer mes del 2023 a poco...,exequiel palacios,es,2023-01-30T16:52:46Z
2,{'content': 'deportes\ngustavo puerta ya no ju...,exequiel palacios,es,2023-01-31T20:41:38Z
3,{'content': 'dólar blue\nalberto fernández\nma...,exequiel palacios,es,2023-02-09T18:32:38Z
4,{'content': 'dólar blue\nalberto fernández\nma...,exequiel palacios,es,2023-02-12T21:13:55Z


## Clean data from patterns

In [62]:
'''
To Dos:
- cut in the beginning: {'content': '
- cut in the end: \nAdvertisement\nAdvertisement\nAdvertisement'}
'''
data_wo_pattern = data_lower.copy()

In [63]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"\\nAdvertisement\\nAdvertisement\\nAdvertisement'\}$", "", str(x)))
#print(data_wo_pattern.iloc[0, 0])


## Clean data from non-textual data

In [64]:
'''
To Dos:
- strip numeric
- strip punctuation
- strip multiple whitespaces
'''
data_stripped = data_wo_pattern.copy()

In [65]:
# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_stripped['data'] = data_stripped['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_stripped['data'] = data_stripped['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_stripped['data'] = data_stripped['data'].apply(strip_multiple_whitespaces)

data_stripped.head()

,data,player,language,publishedAt
0,content directo nmercado de fichajes de fútbo...,exequiel palacios,es,2023-01-29T18:25:03Z
1,con el primer mes del a poco de concluir y la ...,exequiel palacios,es,2023-01-30T16:52:46Z
2,deportes ngustavo puerta ya no jugará en el ba...,exequiel palacios,es,2023-01-31T20:41:38Z
3,dólar blue nalberto fernández nmauricio macri ...,exequiel palacios,es,2023-02-09T18:32:38Z
4,dólar blue nalberto fernández nmauricio macri ...,exequiel palacios,es,2023-02-12T21:13:55Z


## General Preprocessing is done at this Point so the data can be stored in a File which then can be Preprocessed individually

In [66]:
#data_stripped.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/data_files/all_data_v3_PreprocessedGeneral.csv')

## Clean data from stopwords

In [20]:
'''QUESTION for us: shall we decide to keep stop words? Or keep some?'''

#nltk
stop_words = nltk.corpus.stopwords.words('english') 
print(sorted(stop_words),'\n', len(stop_words))

# gensim
print(sorted(STOPWORDS),'\n', len(STOPWORDS))

# Create a list of stop words to remove 
stop_words_to_remove = ["aren't",'isn', "isn't",'mightn', "mightn't",'mustn', "mustn't",'needn', "needn't",'no', 'nor', 'not',"shan't",'shouldn', "shouldn't",'wasn', "wasn't","weren't",'wouldn', "wouldn't"] 
'''what are we missing?'''

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  stop_words.remove(word)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [21]:
'''depending on what we decided?'''

'''data_rm = data_stripped.copy()'''

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_wostop['data'] = data_rm['data'].apply(remove_stopwords_from_text)

NameError: name 'data_rm' is not defined

## Clean data from too short words 

In [22]:
'''Anpassung nötig'''
data_noshortwords = data_stripped.copy() #or
'''data_noshortwords = data_wostop.copy()
'''

# strip_short deletes words smaller 3
data_noshortwords['data'] = data_noshortwords['data'].apply(strip_short)

data_noshortwords['data']

0      manchester xaunited xaimploded seville the eur...
1      slick sevilla punish slapdash man united ease ...
2      manchester united imploded seville the europa ...
3      bayer leverkusen has reached the semifinal eur...
4      leverkusen players celebrate after their team ...
                             ...                        
440    content nick emms for mailonline npublished bs...
441    arsenal have released statement criticise the ...
442    content will griffee for mailonline npublished...
443    untold arsenal supporting the club the manager...
444    content editions nen football chelsea nowners ...
Name: data, Length: 445, dtype: object

## Comparing the results of stemming, lemmatization and tokenization

In [23]:
# stem_text transforms the data into root form w/o using a dictionary by certain rules -> Return lowercase and (porter-)stemmed version of string text.
data_stem = data_noshortwords['data'].apply(stem_text)
data_stem = data_stem.tolist()
data_stem

['manchest xaunit xaimplod sevil the europa leagu thursdai while juventu complet good even for the club hold awai sport and advanc the semi final the quarter final second leg xaunit xasuccumb sevilla out juventu hung draw with sport lisbon advanc aggreg while bayer leverkusen won belgium elimin union saint gillois jose mourinho roma beat feyenoord after extra time advanc unit who led two goal with six minut left the first leg befor conced pair late own goal complet the collaps spain sponsor vuuklegoalkeep david gea had disastr night lacklustr xaunit xawer dispatch the six time competit winner after eight minut gea roll pass isol harri maguir scorer on the first leg own goal thi time ambush three oppon gave the ball awai and youssef nesyri accept the present sevilla domin and could have score again befor centr back loic bade head onto hi own shoulder and loop the ball over gea the minut the minut gea charg out but slice drop ball fell nesyri who curl into the empti net complet the victo

In [ ]:
data_lem = data_noshortwords.copy()

#en_model = en_core_web_sm.load()


def apply_lemmatization(text):
    # Parse the text with spaCy
    doc = nlp(text)
    # Extract the lemmas for each token
    lemmas = [token.lemma_ for token in doc]
    # Join the lemmas with a space separator
    return " ".join(lemmas)

# Apply the lemmatization function to the 'text' column using the apply method
data_lem['lemmatized_text'] = data_lem['data'].apply(apply_lemmatization)
data_lem

: 

In [ ]:
data_bert = data_noshortwords.copy() 

# define bert tokenizer
bert_uncased = BertTokenizer.from_pretrained('bert-base-uncased')

# print out tokens
print(list(bert_uncased.vocab.keys())[1997:2100])

: 

In [ ]:
data_bert = data_noshortwords['data'] 

# convert into list 
data_bert = data_bert.tolist()

# run tokenizer over all list objects
data_BERT = [bert_uncased.tokenize(text) for text in data_bert]
data_BERT[0]

: 